<a href="https://colab.research.google.com/github/jmelsbach/ai-im/blob/main/notebooks/Exercises/01%20-%20Neural%20Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A simple Neural Network with Pytorch
PyTorch is an open-source machine learning framework and is widely used in research and in production.

In this notebook you will learn how to create a Neural Network with Pytorch that is able classify hand-written digits and does this with an accuracy of over 97%.

In [ ]:
import torch
import torch.nn as nn
import numpy as np

There are four required building blocks when it comes to training a neural network:
* Data
* Model
* Loss Function 
* Optimizer

## Data
First of all we need data to train our network on. In the real world collecting and processing data is a very time consuming process. Luckily for us the research community published a lot of high quality datasets in the past that are ready to use for us.

In this notebook we will use the famous MNIST Dataset which consists of several thousand instances of handwritten digits with their coresponding labels.

 We download the dataset via the sklearn library. So we need to import sklearn and download the files

In [ ]:
from sklearn.datasets import fetch_openml

In [ ]:
%%time
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

 We downloaded the dataset and saved created the variables `X` containing the pictures and `y` containing the labels. Let's have a closer look at the data.

In [ ]:
y.shape

`.shape` is a very useful command to see the dimensions of a vector or matrix. As we can see the shape of y is `(70000,)` which basically means it is a vector with 70000 rows. This means that we have 70000 labels and therefore 70000 examples in our dataset. Let's have a look at the first label:

In [ ]:
y[0]

Next we look at the pictures of the digits which are stored in the `X` variable.

In [ ]:
X.shape

The first dimension looks familiar. We had 70000 labels so it definitly makes sense to have 70000 images as well. You might have expected a three dimensional variable here, because images are a matrix of pixels. In this dataset the pictures are flattend into vectors with 784 dimensions. The original size of the images are 28x28 Pixels. 

If we want to look at the first picture of the dataset we have to reshape the images into a 28 by 28 matrix. We use `mathplotlib.pyplot.imshow()` to visualize the image.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x0 = X[0]
x0 = x0.reshape([28,28])
x0.shape

As you can see above currently the values range from 0 to 255. In Deep Learning we almost always want to have our input values to range from 0 to 1 instead. Scaling the data is very easy:

In [ ]:
X = X / 255.0

In [ ]:
plt.imshow(x0.reshape(28,28), cmap='Greys')
# print the corresponding label
print("Label: ", y[0])

### Datasets and Dataloaders

In [ ]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
y = y.astype('int8')

In [ ]:
class MNISTDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X).float() / 255.
        self.y = torch.tensor(y).long()

    def __len__(self):
        return len(self.y)
  
    def __getitem__(self, index):
        return (self.X[index], self.y[index])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
mnist_train = MNISTDataset(X_train, y_train)

In [ ]:
mnist_test = MNISTDataset(X_test, y_test)

In [ ]:
train_dl = DataLoader(mnist_train, batch_size=128) 

In [ ]:
test_dl = DataLoader(mnist_test, batch_size=128)

## The Neural Network

In [ ]:
import torch.nn.functional as F

In [ ]:
class NeuralNetworkV1(nn.Module):
    def __init__(self):
        super(NeuralNetworkV1, self).__init__()
        self.l1 = nn.Linear(784,500)
        self.l2 = nn.Linear(500, 10)

    def forward(self, X):
        X = F.relu(self.l1(X))
        X = self.l2(X)
        return X

In [ ]:
net1 = NeuralNetworkV1()

In [ ]:
%%time
output = net1(mnist_train[0][0])
print(output)
print(output.shape)

This was one forward pass through our neural network. We get an output of shape `[10]` but of quite random positive and negative numbers.

Instead of random numbers we want our network to output a probability distribution, where hopefully the correct number has the highest probability. We have to modify our output in a way that it outputs probabilities, which implies to properties:
1. Each output *must* be between `0` and `1`
2. All outputs have to add up to `1` in total


💭 Think a moment why all outputs have to add up to `1` in our the digit classification example.

In [ ]:
def softmax(x):
    return x.exp() / x.exp().sum()

In [ ]:
class NeuralNetworkV2(nn.Module):
    def __init__(self):
        super(NeuralNetworkV2, self).__init__()
        self.l1 = nn.Linear(784,200)
        self.l2 = nn.Linear(200, 10)


    def forward(self, X):
        X = F.sigmoid(self.l1(X))
        X = self.l2(X)
        
        return X

$$
\text{Softmax}(x_{i}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}
$$

In [ ]:
net2 = NeuralNetworkV2()

In [ ]:
%%time
output = net2(torch.tensor(X[0]).float() / 256)
output

As you can see we reached our goal. Every output is a number between zero and one and all outputs add up to 1. However, all outputs probabilities are about the same at about `0.1`. Which basically means that our network has no idea which digit is shown here. And how should it? It hasn't learned anything yet.

## Loss Function

We need a way to see how good or how bad our neural net is doing. When you take a look in the documentation of the `nn.CrossEntropyLoss()`

In [ ]:
loss_func = nn.CrossEntropyLoss()

## Optimizer

In [ ]:
from torch import optim

In [ ]:
optimizer = optim.Adam(net2.parameters(), lr=0.001)

## Training Loop

In [ ]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [ ]:
epochs = 100

In [ ]:
for epoch in range(epochs):
    net2.train()
    total_loss = 0
    train_acc = 0
    for inputs, labels in train_dl:
        
        preds = net2(inputs)
        loss = loss_func(preds, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss
        train_acc += accuracy(preds, labels)
        
    
    net2.eval()
    train_acc /= len(train_dl)
    print(f"train_acc: {train_acc}")
    #print(f"Train Loss: {total_loss}")
    with torch.no_grad():
        val_acc = 0
        for inputs_test, labels_test in test_dl:
            val_acc += accuracy(net2(inputs_test), labels_test)
        val_acc /= len(test_dl)
        print(f"val_acc: {val_acc}")

In [ ]:
accuracy(net2(mnist_test[0:8][0]), mnist_test[0:8][1])

In [ ]:
mnist_test[0:8][0]

In [ ]:
mnist_test[0:8][1]

In [ ]:
len(test_dl)